# PlayPulse Steam Metrics

This project explores the Steam API to gather and analyze player count data for various games. The goal is to understand player engagement trends and provide insights using the collected data.

In [61]:
import pandas as pd
import requests
import os
import psycopg2
from dotenv import load_dotenv
import time


Start small with getting app list.

In [62]:
url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
response = requests.get(url)

data = response.json()

df = pd.DataFrame(data['applist']['apps'])

print(df.head())

print(f'Number of games: {len(df)}')

   appid                   name
0      5       Dedicated Server
1      7           Steam Client
2      8                 winui2
3     10         Counter-Strike
4     20  Team Fortress Classic
Number of games: 265258


In [63]:
# Example: Get list of games from Steam API

def get_steam_games():
    """
    Fetches a list of games from the Steam API.
    Returns:
        list: List of games, or None if request fails.
    """
    url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get('applist', {}).get('apps', [])
    else:
        print(f"Failed to fetch data: {response.status_code}")

get_steam_games()

[{'appid': 5, 'name': 'Dedicated Server'},
 {'appid': 7, 'name': 'Steam Client'},
 {'appid': 8, 'name': 'winui2'},
 {'appid': 10, 'name': 'Counter-Strike'},
 {'appid': 20, 'name': 'Team Fortress Classic'},
 {'appid': 30, 'name': 'Day of Defeat'},
 {'appid': 40, 'name': 'Deathmatch Classic'},
 {'appid': 50, 'name': 'Half-Life: Opposing Force'},
 {'appid': 60, 'name': 'Ricochet'},
 {'appid': 70, 'name': 'Half-Life'},
 {'appid': 80, 'name': 'Counter-Strike: Condition Zero'},
 {'appid': 90, 'name': 'Half-Life Dedicated Server'},
 {'appid': 92, 'name': 'Codename Gordon'},
 {'appid': 100, 'name': 'Counter-Strike: Condition Zero Deleted Scenes'},
 {'appid': 130, 'name': 'Half-Life: Blue Shift'},
 {'appid': 150, 'name': 'Counter-Strike Steamworks Beta'},
 {'appid': 205, 'name': 'Source Dedicated Server'},
 {'appid': 210, 'name': 'Source Dedicated Server'},
 {'appid': 211, 'name': 'Source SDK'},
 {'appid': 215, 'name': 'Source SDK Base 2006'},
 {'appid': 218, 'name': 'Source SDK Base 2007'},
 {

In [64]:
df = pd.DataFrame(get_steam_games())

In [65]:
df

,appid,name
0,5,Dedicated Server
1,7,Steam Client
2,8,winui2
3,10,Counter-Strike
4,20,Team Fortress Classic
...,...,...
265253,4017560,Marshals of Yore Demo
265254,4017640,Sahir: Shiv of Memories Demo
265255,4020440,Overkill Squad Demo
265256,4020900,Lily's Erotic Potion Delivery Demo


In [66]:
df.to_csv('steam_games.csv', index=False)

In [68]:
appid=730 # Counter-Strike: Global Offensive
url=f"https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/?appid={appid}"

response=requests.get(url)
data=response.json()
data


{'response': {'player_count': 613399, 'result': 1}}

In [69]:
# Get current player count for a Steam app using the Steam Web API

def get_steam_player_count(appid):
    """
    Fetches the current number of players for a given Steam app ID.
    Args:
        appid (int): The Steam App ID of the game.
    Returns:
        int: Current number of players, or None if request fails.
    """
    url = f"https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/?appid={appid}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get('response', {}).get('player_count')
    else:
        print(f"Failed to fetch data for appid {appid}: {response.status_code}")
        return None

# Example usage:
appid = 730  # Counter-Strike: Global Offensive
print(get_steam_player_count(appid))


613399


In [70]:
# game list

game_list = {
    "csgo": 730,
    "halo mcc": 976730,
    "halo infinite": 1240440,
    "dota 2": 570,
    "overwatch 2": 2357570,
    "apex legends": 1172470,
    "marvel rivals": 2767030,
    "hollow knight: silksong": 1030300
}


In [ ]:
rows = []

for game, appid in game_list.items():
    player_count = get_steam_player_count(appid)
    rows.append({'appid': appid, 'game': game, 'player_count': player_count})
    
current_players = pd.DataFrame(rows)
print(current_players)

     appid                     game  player_count
0      730                     csgo        613399
1   976730                 halo mcc          4636
2  1240440            halo infinite          3244
3      570                   dota 2        445557
4  2357570              overwatch 2         28234
5  1172470             apex legends         51543
6  2767030            marvel rivals         75967
7  1030300  hollow knight: silksong        258079


## Storing Data
I'm going to try to use an open source platform like Supabase for storage

In [72]:
from supabase import create_client
import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_API_KEY")

if SUPABASE_URL and SUPABASE_KEY:
    supabase = create_client(SUPABASE_URL, SUPABASE_KEY)
    print("Connected to Supabase!")
else:
    print("Supabase credentials not found in environment variables.")


Connected to Supabase!


In [75]:
insert_rows = (current_players.to_dict(orient='records'))

response = supabase.table('player_counts').insert(insert_rows).execute()
print("Inserted rows:", response)

Inserted rows: data=[{'created_at': '2025-09-10T04:09:01.070178+00:00', 'id': 1, 'appid': 730, 'game': 'csgo', 'player_count': 613399}, {'created_at': '2025-09-10T04:09:01.070178+00:00', 'id': 2, 'appid': 976730, 'game': 'halo mcc', 'player_count': 4636}, {'created_at': '2025-09-10T04:09:01.070178+00:00', 'id': 3, 'appid': 1240440, 'game': 'halo infinite', 'player_count': 3244}, {'created_at': '2025-09-10T04:09:01.070178+00:00', 'id': 4, 'appid': 570, 'game': 'dota 2', 'player_count': 445557}, {'created_at': '2025-09-10T04:09:01.070178+00:00', 'id': 5, 'appid': 2357570, 'game': 'overwatch 2', 'player_count': 28234}, {'created_at': '2025-09-10T04:09:01.070178+00:00', 'id': 6, 'appid': 1172470, 'game': 'apex legends', 'player_count': 51543}, {'created_at': '2025-09-10T04:09:01.070178+00:00', 'id': 7, 'appid': 2767030, 'game': 'marvel rivals', 'player_count': 75967}, {'created_at': '2025-09-10T04:09:01.070178+00:00', 'id': 8, 'appid': 1030300, 'game': 'hollow knight: silksong', 'player_co